### Introduction to Machine Learning
##### https://keras.io/getting-started/sequential-model-guide/
#### prepared by Chao-Lin Liu
###### 目前這一個程式的辨識率很差
###### 改變一下你的 sequential model 的設定，看看能不能達成比較高的辨識率？
###### 能不能把辨識率提高到 90% 以上？

In [3]:
import tensorflow as tf

In [4]:
import pandas as pd
import numpy as np
df = pd.read_csv("practice16.train.csv")

In [5]:
df[0:5]

,f0,f1,f2,class
0,0.633209,0.617612,0.163622,1
1,0.812224,0.241355,0.069014,1
2,0.638819,0.938740,0.834770,2
3,0.600864,0.603507,0.616036,2
4,0.075439,0.852837,0.337803,1


In [6]:
X_train = np.array(df.iloc[:,0:3])
y_train = np.array(df.iloc[:,3:4]).reshape(-1)

In [7]:
X_train

array([[0.63320928, 0.61761157, 0.16362248],
       [0.81222375, 0.24135473, 0.06901366],
       [0.63881877, 0.93874007, 0.83477015],
       ...,
       [0.51401413, 0.25990072, 0.36776594],
       [0.25605967, 0.6596416 , 0.71476801],
       [0.60072969, 0.58385003, 0.48550142]])

In [8]:
#shuffle data
from sklearn.utils import shuffle
X_train, y_train = shuffle(X_train, y_train)

In [9]:
# 以下的步驟在 Introduction to Machine learning 課程並沒有講到
# 我們把 y 的數值變成了向量 － one-hot vector
#
from sklearn.preprocessing import LabelEncoder
# from keras.utils import np_utils
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(y_train)
encoded_npy = encoder.transform(y_train)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_npy = tf.keras.utils.to_categorical(encoded_npy)

In [10]:
y_train[0:5]

array([1, 2, 2, 1, 0], dtype=int64)

In [11]:
dummy_npy[0:5]

array([[0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.]], dtype=float32)

In [12]:
# 你可以試著增加 Dense 層的數目、或者改變個別 Dense 層的 nodes 數目、或者改變 activation functions
#
# from keras.models import Sequential
# from keras.layers import Dense
# define baseline model
def baseline_model():
    # create model
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(5, input_dim=3, activation='relu'))
    model.add(tf.keras.layers.Dense(5, activation='relu'))
    model.add(tf.keras.layers.Dense(4, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [13]:
model2 = baseline_model()

In [19]:
len(X_train)

3000

In [20]:
y_train[0:5]

array([1, 2, 2, 1, 0], dtype=int64)

In [ ]:
# 在這裡，你可以嘗試增減 epochs 和 batch_size
model2.fit(X_train , dummy_npy, epochs=50, batch_size=20, verbose=1, shuffle=True)

In [16]:
df2 = pd.read_csv("practice16.test.csv")

In [17]:
testX = np.array(df2.iloc[:,0:3])

In [18]:
predicty = model2.predict(testX)

InternalError:  Blas GEMM launch failed : a.shape=(32, 3), b.shape=(3, 5), m=32, n=5, k=3
	 [[node sequential/dense/MatMul (defined at <ipython-input-18-c11cb2adfee2>:1) ]] [Op:__inference_predict_function_177]

Function call stack:
predict_function


In [ ]:
predicty[0:5]

In [49]:
testy = np.array(df2.iloc[:,3:4]).reshape(-1)

In [50]:
predicty = encoder.inverse_transform([np.argmax(item) for item in predicty])

In [51]:
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(testy,predicty))

[[  0  10   0   0]
 [  0 238  11   0]
 [  0   1 229   0]
 [  0   0  11   0]]


In [52]:
print(classification_report(testy,predicty))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        10
           1       0.96      0.96      0.96       249
           2       0.91      1.00      0.95       230
           3       0.00      0.00      0.00        11

    accuracy                           0.93       500
   macro avg       0.47      0.49      0.48       500
weighted avg       0.90      0.93      0.91       500



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [53]:
predicty[0:5]

array([1, 1, 2, 1, 1], dtype=int64)

In [54]:
len(predicty)

500

In [55]:
togo = open('practice16.output.csv', "w")
togo.write('id,Predicted\n')
for index in range(0,len(predicty)):
    togo.write(str(index+1)+","+str(predicty[index])+'\n')
togo.close()